#Algorithme de Berstein-Vazirani

Imports

In [ ]:
def OracleBersteinVazirani(n_qubits, bits_string):
    string_length = len(string)

    qc = QuantumCircuit(string_length + 1, string_length)

    for i in range(string_length):
        qc.h(i)

    qc.x(string_length)
    qc.h(string_length)

    string = string[::-1]

    for i in range(string_length):
        if string[i] == '1':
            qc.cx(i, string_length)

    for i in range(string_length):
        qc.h(i)

    qc.measure(range(string_length), range(string_length))

    return qc